***This code is used to clean, combine, and output the necessary .csv files with all of the data necessary for analyses computed in Intrator's "Effects of Health and Economic Crisis on Healthcare Spending in the United States".***

* data collected from OECD are saved into a single dataframe
* other data from BLS, CDC are saved in separate dataframes
* all cleaned data are imported and brought together in the analyses notebook

In [1]:
## package imports
import pandas as pd
import numpy as np

# Importing and cleaning all datasets

In [2]:
## initializing a list to which all the dataframes will be appending and that will eventually become the final dataset
data_list = []

## health spending variables

In [3]:
## importing original health dataset
health = pd.read_csv('oecd health.csv')

## creating a temporary list of pandas dataframes that will be joined together at the end
health_list = []

## extracting relevant response variables and putting them into multiple columns instead of one extended column
for sub in health.SUBJECT.unique():
    for measure in health.MEASURE.unique():
        if measure == 'USD_CAP':
            temp = health[(health.SUBJECT==sub) & (health.MEASURE==measure)][['TIME', 'Value']]
            temp.columns = ['year', sub+'_'+measure]
            temp = temp.set_index('year')
            health_list.append(temp)
            
health_data = health_list[0].join(health_list[1:])

data_list.append(health_data)

## alcohol consumption

In [4]:
## importing relevant columns from original dataset
alc = pd.read_csv('alcohol.csv', usecols=['TIME', 'Value'])
alc.columns = ['year', 'alc']
alc = alc.set_index('year')

data_list.append(alc)

## unemployment

In [5]:
## importing relevant columns from original dataset
unemp = pd.read_csv('unemployment.csv', usecols=['TIME', 'Value'])
unemp.columns = ['year', 'unemp']
unemp = unemp.set_index('year')

data_list.append(unemp)

## elderly

In [6]:
## importing relevant columns from original dataset
elderly = pd.read_csv('elderly.csv', usecols=['TIME', 'Value'])
elderly.columns = ['year', 'elderly']
elderly = elderly.set_index('year')

data_list.append(elderly)

## life expectancy

In [7]:
## importing relevant columns from original dataset
life_exp = pd.read_csv('life expectancy.csv', usecols=['TIME', 'Value'])
life_exp.columns = ['year', 'life_exp']
life_exp = life_exp.set_index('year')

data_list.append(life_exp)

## household disposable income

In [8]:
## importing relevant columns from original dataset
income = pd.read_csv('HHDI.csv', usecols=['TIME', 'Value'])
income.columns = ['year', 'income']
income = income.set_index('year')

data_list.append(income)

## life expectancy at 65

In [9]:
## importing relevant columns from original dataset

## life expectancy of men at 65
life_exp65m = pd.read_csv('life_exp at 65 men.csv', usecols=['TIME', 'Value'])
life_exp65m.columns = ['year', 'life_exp65m']
life_exp65m = life_exp65m.set_index('year')

## life expectancy of women at 65
life_exp65w = pd.read_csv('life_exp at 65 women.csv', usecols=['TIME', 'Value'])
life_exp65w.columns = ['year', 'life_exp65w']
life_exp65w = life_exp65w.set_index('year')

## averaging out the life expectancy at 65 between men and women each year
life_exp65 = pd.DataFrame(life_exp65m.join(life_exp65w).mean(axis=1))
life_exp65.columns = ['life_exp65']

data_list.append(life_exp65)

## HIV/AIDS crude death rate ((deaths/population) * 100k)

In [10]:
## importing and cleaning the relevant .csv's
aids98 = pd.read_csv('AIDS deaths - 1979-1998 (WONDER Archive).csv', usecols = ['Year', 'Crude Rate']).dropna()
aids98.columns = ['year', 'aids_crude']
aids98 = aids98.set_index('year')

## cleaning the initial years so the '(Unreliable)' is not part of the number anymore for aids98
for yr in range(1979, 1987):
    aids98.loc[yr, 'aids_crude'] = float(aids98.loc[yr, 'aids_crude'][:7])

aids15 = pd.read_csv('AIDS deaths - 1999-2015 (WONDER Archive).csv', usecols = ['Year', 'Crude Rate']).dropna()
aids15.columns = ['year', 'aids_crude']
aids15 = aids15.set_index('year')

## appending the datasets together
aids = aids98.append(aids15)

## appending 0s for the crude death rate from 1970-1978 so that we have complete data for other years
yrs_without_aids = pd.DataFrame(
    {'year': list(range(1970,1979)), 'aids_crude': [0]*len(range(1970,1979))}
    ).set_index('year')

aids = yrs_without_aids.append(aids)

## adding in a dummy variable for the years with reliable HIV/AIDS data (namely years 1979 - 1986)
aids['reliable'] = [1]*len(aids)
for yr in range(1979, 1987):
    aids.loc[yr, 'reliable'] = 0
    
## adding the interaction term between the aids_crude and the reliable data dummy
aids['aids_rel_inter'] = aids.aids_crude * aids.reliable

## forcing everything in the dataframe to be a float
aids = aids.astype(float)

## saving this aids data to a .csv file to be used in the analyses
aids.to_csv('cleaned aids data - 1970-2015.csv')

## opioid crude death rate ((deaths/population) * 100k)

In [11]:
## data here is in a very weird format, and is also very small, so it will be copied and pasted below
## the file where this data was pulled from can be found in the repository, it is called "opioid Overdose_data_1999-2019.xlsx"
year = list(range(1999,2016))
opioid_crude = [2.9, 3.0, 3.3, 4.1, 4.5, 4.7, 5.1, 5.9, 6.1, 6.4, 6.6, 6.8, 7.3, 7.4, 7.9, 9.0, 10.4]

## creating the opioid pandas.DataFrame
opioid = pd.DataFrame([year, opioid_crude]).transpose()
opioid.columns = ['year', 'opioid_crude']
opioid = opioid.set_index('year')
opioid.index = opioid.index.astype(int)

## exporting data to .csv file for the same reasons that the HIV/AIDS data was exported to a separate file
opioid.to_csv('cleaned opioid data - 1999-2015.csv')

## Importing Medical Care CPI
Will be used as the measure of prices for healthcare in each year (base year = 1982-1984), then we may divide the total spending by the CPI of the same year to have an estimate for the quantity of healthcare in that year (although that is a very weird way of thinking about it, but still)

In [12]:
## importing the actual data and renaming the columns
cpi = pd.read_csv('medical cpi.csv')
cpi.columns = ['year', 'cpi']

## changing the 'year' column to be a single year, then setting the year column as the index and forcing it to be an int
cpi.year = [int(yr[:4]) for yr in cpi.year]
cpi = cpi.set_index('year')
cpi.index = cpi.index.astype(int)

## cutting off data that is irrelevant to the rest of the data for this study - in this case any data past 2013)
cpi = cpi[cpi.index <= 2013]

## forcing the 'cpi' column to be a float
cpi.cpi = cpi.cpi.astype(float)

## saving the cpi data to a csv to be used later
cpi.to_csv('clean medical cpi.csv')

# Combining all datasets and saving

## combining and cleaning full data

In [13]:
full_data = data_list[0].join(data_list[1:])

In [14]:
## removing all null values and cleaning so all data is from before the Affordable Care Act (up to 2013)
full_data = full_data.dropna()
full_data = full_data.loc[full_data.index <= 2013]

In [15]:
## reordering the columns to be presented better
cols = [
    'TOT_USD_CAP', 'OOPEXP_USD_CAP', 'COMPULSORY_USD_CAP', 'VOLUNTARY_USD_CAP', # dependent variables
    'unemp', 'income', # economic-related independent variables
    'alc', 'elderly', 'life_exp', 'life_exp65' # health-related independent variables
]

## reordering the variables in the final dataframe to help keep track better
full_data = full_data[cols]

In [16]:
full_data

,TOT_USD_CAP,OOPEXP_USD_CAP,COMPULSORY_USD_CAP,VOLUNTARY_USD_CAP,unemp,income,alc,elderly,life_exp,life_exp65
year,,,,,,,,,,
1970,326.961,121.691,121.618,205.343,4.933333,4105.495951,9.5,9.806,70.9,15.05
1971,357.988,126.707,136.449,221.539,5.958333,4436.848815,9.8,9.901,71.2,15.05
1972,397.097,136.477,151.828,245.269,5.616667,4771.307640,9.6,10.014,71.3,15.05
1973,439.302,149.239,170.724,268.578,4.891667,5284.992031,9.8,10.158,71.5,15.15
1974,495.114,160.731,200.048,295.066,5.591667,5748.870990,10.0,10.316,72.1,15.45
1975,560.750,172.531,233.112,327.638,8.466666,6341.296758,10.1,10.509,72.7,15.85
1976,638.851,186.424,261.985,376.866,7.716667,6840.435104,10.1,10.676,73.0,15.85
1977,726.241,203.656,294.325,431.916,7.066667,7442.221745,9.8,10.848,73.4,16.10
1978,808.884,215.720,331.748,477.136,6.066667,8217.869536,10.1,11.008,73.5,16.20


## Saving full dataset to a .csv

In [17]:
full_data.to_csv('clean dataset.csv')